In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


In [2]:
height,width=32,32
batch_size=64

In [3]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

# Define the augmentation probabilities
p_dict = {
  # "flip": 0.5,
  # "transpose": 0.5,
  # "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}

p_dicts = []
for aug_type in p_dict.keys():
    p_d = {aug_type: .5}
    p_dicts.append(p_d)

p_dict = {
  "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}
p_dicts.append(p_dict)

p_dicts


[{'flip': 0.5, 'gauss_noise': 0.5}]

In [4]:
import matplotlib.pyplot as plt

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [5]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.kerasCNN import *
import pickle


EPOCHS = 50

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(
            kernel_sizes=[3, 3, 3,],
            num_filters=[32, 64, 64],
            fc_sizes=[128, 64, 64],
        )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/BESTClassicCNN_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/1 [00:00<?, ?it/s]

Augmentations: flip0.5_gauss_noise0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch 1/50
1336/1336 [==============================] - 25s 18ms/step - loss: 1.6362 - accuracy: 0.3943 - val_loss: 1.6682 - val_accuracy: 0.4000
Epoch 2/50
1336/1336 [==============================] - 22s 16ms/step - loss: 1.3540 - accuracy: 0.5100 - val_loss: 1.6197 - val_accuracy: 0.4358
Epoch 3/50
1336/1336 [==============================] - 25s 19ms/step - loss: 1.2679 - accuracy: 0.5464 - val_loss: 1.2926 - val_accuracy: 0.5291
Epoch 4/50
1336/1336 [==============================] - 23s 17ms/step - loss: 1.1935 - accuracy: 0.5731 - val_loss: 1.2897 - val_accuracy: 0.5518
Epoch 5/50
1336/1336 [==============================] - 24s 18ms/step - loss: 1.1461 - accuracy: 0.5929 - val_loss: 1.2459 - val_accuracy: 0.5427
Epoch 6/50
1336/1336 [==============================] - 23s 17ms/step - loss: 1.0780 - accuracy: 0.6152 - val_loss: 1.1581 - val_accuracy: 0.5862
Epoch 7/50
1336/1336 [==============================] - 24s 18ms/step - loss: 1.0290 - accuracy: 0.6351 - val_loss: 1.1391 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 23s 17ms/step - loss: 1.6332 - accuracy: 0.3963 - val_loss: 1.5529 - val_accuracy: 0.4484
Epoch 2/50
1336/1336 [==============================] - 22s 16ms/step - loss: 1.3582 - accuracy: 0.5076 - val_loss: 1.8286 - val_accuracy: 0.3673
Epoch 3/50
1336/1336 [==============================] - 22s 16ms/step - loss: 1.2367 - accuracy: 0.5543 - val_loss: 1.3337 - val_accuracy: 0.5302
Epoch 4/50
1336/1336 [==============================] - 22s 16ms/step - loss: 1.1599 - accuracy: 0.5852 - val_loss: 1.4480 - val_accuracy: 0.5029
Epoch 5/50
1336/1336 [==============================] - 24s 18ms/step - loss: 1.1743 - accuracy: 0.5817 - val_loss: 1.2663 - val_accuracy: 0.5487
Epoch 6/50
1336/1336 [==============================] - 24s 18ms/step - loss: 1.0998 - accuracy: 0.6077 - val_loss: 1.2627 - val_accuracy: 0.5531
Epoch 7/50
1336/1336 [==============================] - 22s 16ms/step - loss: 1.0726 - accuracy: 0.6199 - val_loss: 1.2386 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 26s 19ms/step - loss: 1.6299 - accuracy: 0.4001 - val_loss: 1.5042 - val_accuracy: 0.4538
Epoch 2/50
1336/1336 [==============================] - 25s 18ms/step - loss: 1.3372 - accuracy: 0.5172 - val_loss: 1.3555 - val_accuracy: 0.5133
Epoch 3/50
1336/1336 [==============================] - 24s 18ms/step - loss: 1.2253 - accuracy: 0.5633 - val_loss: 1.2071 - val_accuracy: 0.5687
Epoch 4/50
1336/1336 [==============================] - 23s 17ms/step - loss: 1.2055 - accuracy: 0.5703 - val_loss: 1.2839 - val_accuracy: 0.5373
Epoch 5/50
1336/1336 [==============================] - 23s 17ms/step - loss: 1.1250 - accuracy: 0.5990 - val_loss: 1.3437 - val_accuracy: 0.5271
Epoch 6/50
1336/1336 [==============================] - 25s 18ms/step - loss: 1.1023 - accuracy: 0.6092 - val_loss: 1.3942 - val_accuracy: 0.5320
Epoch 7/50
1336/1336 [==============================] - 25s 18ms/step - loss: 1.0442 - accuracy: 0.6279 - val_loss: 1.1244 -